In [2]:
import pandas as pd
import numpy as np


In [3]:

trip_data=pd.read_csv("Trip-Data-for-ML-Model.csv")

trip_data


,Operated_Day,Route_Name,Actual_Trip_Start,Actual_Trip_End
0,Tuesday,801AUp,4/30/2024 13:19,4/30/2024 13:49
1,Tuesday,801Down,4/30/2024 15:09,4/30/2024 16:28
2,Tuesday,801LnkSTLDown,4/30/2024 19:30,4/30/2024 20:31
3,Tuesday,772ASTLUp,4/30/2024 12:47,4/30/2024 12:53
4,Tuesday,801ADown,4/30/2024 11:54,4/30/2024 12:37
...,...,...,...,...
806686,Monday,109Up,4/1/2024 8:18,4/1/2024 10:16
806687,Monday,109ADown,4/1/2024 19:57,4/1/2024 21:01
806688,Monday,109AUp,4/1/2024 21:06,4/1/2024 21:52
806689,Monday,109Down,4/1/2024 14:41,4/1/2024 16:53


In [3]:
# trip_data['Actual_Trip_Start'] = pd.to_datetime(trip_data['Actual_Trip_Start'], format="%H:%M:%S")
# trip_data['Actual_Trip_End'] = pd.to_datetime(trip_data['Actual_Trip_End'], format="%H:%M:%S")


In [6]:
trip_data['Actual_Trip_Start'] = pd.to_datetime(trip_data['Actual_Trip_Start'], format="%m/%d/%Y %H:%M")
trip_data['Actual_Trip_End'] = pd.to_datetime(trip_data['Actual_Trip_End'], format="%m/%d/%Y %H:%M")
trip_data['Actual_Trip_Start']=trip_data['Actual_Trip_Start'].dt.time
trip_data['Actual_Trip_End']=trip_data['Actual_Trip_End'].dt.time

In [7]:
def time_to_float(t):

    return t.hour + t.minute / 60 + t.second / 3600

trip_data['Actual_Trip_Start']=trip_data['Actual_Trip_Start'].apply(time_to_float)

trip_data['Actual_Trip_End']=trip_data['Actual_Trip_End'].apply(time_to_float)

In [8]:
trip_data['Operated_Day'].unique()

array(['Tuesday', 'Monday', 'Sunday', 'Saturday', 'Friday', 'Thursday',
       'Wednesday'], dtype=object)

In [9]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder() 
label_encoder.fit(trip_data['Route_Name'].unique())

import joblib
# Save the LabelEncoder to a file
joblib.dump(label_encoder, 'label_encoder.pkl')


trip_data['Route_Name'] = label_encoder.transform(trip_data['Route_Name'])

day_mapping = {"Monday": 0, "Tuesday": 1, "Wednesday": 2, "Thursday": 3,"Friday": 4, "Saturday": 5, "Sunday": 6}
trip_data['Operated_Day'] = trip_data['Operated_Day'].map(day_mapping)
trip_data

,Operated_Day,Route_Name,Actual_Trip_Start,Actual_Trip_End
0,1,820,13.316667,13.816667
1,1,821,15.150000,16.466667
2,1,824,19.500000,20.516667
3,1,762,12.783333,12.883333
4,1,819,11.900000,12.616667
...,...,...,...,...
806686,0,41,8.300000,10.266667
806687,0,36,19.950000,21.016667
806688,0,37,21.100000,21.866667
806689,0,38,14.683333,16.883333


In [10]:
Features = ['Actual_Trip_Start','Route_Name','Operated_Day']
X=trip_data[Features]
y = trip_data['Actual_Trip_End']

X = X.astype(np.float32)
y = y.astype(np.float32)


In [11]:
# linear regresssion

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

trip_model= LinearRegression()
trip_model.fit(X_train, y_train)

LinearRegression()

In [12]:
y_pred = trip_model.predict(X_test)
print(y_pred)
# print(((y_pred>24)==True).sum())
# if (y_pred>24).any(): print(y_pred[y_pred>24])

[23.472654   6.7648787 11.334769  ... 13.703808  10.620318   7.76074  ]


In [13]:
mean_absolute_error(y_test, y_pred)

0.7158893

In [14]:
from sklearn.metrics import r2_score

# For Linear Regression
r2_linear = r2_score(y_test, y_pred)
print("R-squared for Linear Regression:", r2_linear)


R-squared for Linear Regression: 0.8680774950943126


In [13]:
# Random Forest Regressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor()
model.fit(X_train, y_train)


KeyboardInterrupt



In [ ]:
y_pred_rf = model.predict(X_test)
y_pred_rf

In [ ]:
mean_absolute_error(y_test, y_pred_rf)

In [ ]:

# For Random Forest
r2_rf = r2_score(y_test, y_pred_rf)
print("R-squared for Random Forest:", r2_rf)

In [ ]:
from datetime import time

def float_to_time(f):

    hours = int(f)

    minutes = int((f - hours) * 60)

    seconds = int((f - hours - minutes / 60) * 3600)

    return time(hours, minutes, seconds)



In [ ]:
trip_data['Actual_Trip_Start']=trip_data['Actual_Trip_Start'].apply(float_to_time)

trip_data['Actual_Trip_End']=trip_data['Actual_Trip_End'].apply(float_to_time)

In [ ]:
trip_data

In [ ]:

import joblib
from sklearn.ensemble import RandomForestClassifier  # Example

# model = RandomForestClassifier()
# model.fit(X_train, y_train)

# Save the model to a .pkl file
joblib.dump(trip_model, 'tripml_model.pkl')
